In [1]:
!git clone https://github.com/raghav-thiruv/Omdena-Quantum-Self-Driving.git

Cloning into 'Omdena-Quantum-Self-Driving'...
remote: Enumerating objects: 45761, done.
remote: Counting objects: 100% (11671/11671), done.
remote: Compressing objects: 100% (11653/11653), done.
remote: Total 45761 (delta 32), reused 11645 (delta 17), pack-reused 34090
Receiving objects: 100% (45761/45761), 2.16 GiB | 39.25 MiB/s, done.
Resolving deltas: 100% (61/61), done.
Updating files: 100% (45593/45593), done.


In [2]:
!mv "/content/Omdena-Quantum-Self-Driving/Images/driving_dataset1/data.txt" "/content/Omdena-Quantum-Self-Driving/Images/"

In [3]:
# !du -h /content/Omdena-Quantum-Self-Driving

In [4]:
from struct import unpack
from tqdm import tqdm
import os

img_dir = '/content/Omdena-Quantum-Self-Driving/Images/driving_dataset1/'
data_file = "/content/Omdena-Quantum-Self-Driving/Images/data.txt"

In [5]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Input, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model

In [14]:
def build_model():
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    for layer in base_model.layers:
        layer.trainable = True

    flatten_output = Flatten()(base_model.output)

    angle_input = Input(shape=(1,))
    fused_output = Concatenate()([flatten_output, angle_input])

    fc1 = Dense(512, activation='relu')(fused_output)
    # dropout1 = Dropout(0.3)(fc1)
    fc2 = Dense(256, activation='relu')(fc1)
    output = Dense(1)(fc2)

    model = Model(inputs=[base_model.input, angle_input], outputs=output)

    return model


# Load and preprocess a single image
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.vgg19.preprocess_input(image)
    return image

def create_dataset(image_files, angles):
    image_paths = [os.path.join(img_dir, file_name) for file_name in image_files]
    image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    angle_dataset = tf.data.Dataset.from_tensor_slices(angles)
    image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = tf.data.Dataset.zip((image_dataset, angle_dataset))
    return dataset

# Get the list of image files
image_files = []

# Load the steering angles
angles_dict = {}
with open(data_file, 'r') as file:
    for line in file:
        img_file, angle = line.strip().split()
        image_files.append(img_file)
        angles_dict[img_file] = float(angle)

# Get the angles for the image files
angles = [angles_dict[file_name] for file_name in image_files]

# Split the image files into training and validation sets
validation_split = 0.2
num_validation_samples = int(validation_split * len(image_files))
train_image_files = image_files[num_validation_samples:]
train_image_angles = angles[num_validation_samples:]
val_image_files = image_files[:num_validation_samples]
val_image_angles = angles[:num_validation_samples]

# Create the training and validation datasets with both inputs
train_dataset = create_dataset(train_image_files, train_image_angles)
val_dataset = create_dataset(val_image_files, val_image_angles)

# Shuffle and batch the datasets
batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

# Define the input shape for the angle dataset
angle_input_shape = (1,)

# Create the VGG-16 model
model = build_model()

# Define the mean squared error as the loss function
def mse_loss(y_true, y_pred):
    return tf.keras.losses.mean_squared_error(y_true, y_pred)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=mse_loss)

# Modify the fit call to pass both inputs
train_dataset = train_dataset.map(lambda img, angle: ((img, angle), angle))
val_dataset = val_dataset.map(lambda img, angle: ((img, angle), angle))

In [ ]:
# Train the model
epochs = 50
model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)

Epoch 1/50
1136/1136 [==============================] - 670s 580ms/step - loss: 74.3936 - val_loss: 223.4441
Epoch 2/50
1136/1136 [==============================] - 650s 569ms/step - loss: 1.9753 - val_loss: 105.1200
Epoch 3/50
1136/1136 [==============================] - 619s 541ms/step - loss: 0.0260 - val_loss: 98.8649
Epoch 4/50
1136/1136 [==============================] - 619s 541ms/step - loss: 0.1576 - val_loss: 97.2273
Epoch 5/50
1136/1136 [==============================] - 614s 537ms/step - loss: 0.0151 - val_loss: 131.3139
Epoch 6/50
1136/1136 [==============================] - 608s 532ms/step - loss: 4.0675 - val_loss: 75.6942
Epoch 7/50
   3/1136 [..............................] - ETA: 9:12 - loss: 0.0037

In [ ]:

# marker_mapping = {
#     0xffd8: "Start of Image",
#     0xffe0: "Application Default Header",
#     0xffdb: "Quantization Table",
#     0xffc0: "Start of Frame",
#     0xffc4: "Define Huffman Table",
#     0xffda: "Start of Scan",
#     0xffd9: "End of Image"
# }


# class JPEG:
#     def __init__(self, image_file):
#         with open(image_file, 'rb') as f:
#             self.img_data = f.read()

#     def decode(self):
#         data = self.img_data
#         while(True):
#             marker, = unpack(">H", data[0:2])
#             # print(marker_mapping.get(marker))
#             if marker == 0xffd8:
#                 data = data[2:]
#             elif marker == 0xffd9:
#                 return
#             elif marker == 0xffda:
#                 data = data[-2:]
#             else:
#                 lenchunk, = unpack(">H", data[2:4])
#                 data = data[2+lenchunk:]
#             if len(data)==0:
#                raise TypeError("issue reading jpeg file")


# bads = []

# for dirName, subdirList, fileList in os.walk(img_dir):
#     imagesList = fileList
#     for img in tqdm(imagesList):
#       image = os.path.join(img_dir,img)
#       image = JPEG(image)
#       try:
#         image.decode()
#       except:
#         bads.append(img)


# for name in bads:
#   os.remove(os.path.join(img_dir,name))